In [1]:
import os
import numpy as np
import pandas as pd

data_ = {}

for filename in os.listdir("data/Top100Cryptos/"):
    path = os.path.join("data/Top100Cryptos/", filename)
    try:
        name = filename.split(".")[0]
        data_[name] = pd.read_csv(path)
    except:
        pass

num_data = []
for name in data_.keys():
    num_data.append(data_[name].shape[0])
name_list = np.array(list(data_.keys()))[np.argsort(num_data)[::-1]]
big_names = name_list[:10]

data = dict()
for name in big_names:
    data[name] = data_[name]

In [2]:
from rltensor.environments import TradeEnv
from rltensor.configs import eiie_config
from rltensor.agents import EIIE
import tensorflow as tf

env = TradeEnv(data)

conf = dict(
    action_spec={"type": "float", "shape": env.action_dim},
    state_spec={"type": "float", "shape": (env.action_dim, 3)}
)

default_config = eiie_config()
conf.update(default_config)

fit_config = dict(
        t_max= 10000,
        num_max_start_steps=0,
        log_freq=1)
# _fit_config["log_freq"] = 10
# conf["t_learn_start"] = 100
    
tf.reset_default_graph()
eiie = EIIE(env=env, **conf)
eiie.fit(**fit_config)

/home/tom/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Building tensorflow graph...


  0%|          | 29/10000 [00:00<00:35, 283.85it/s]

Finished building tensorflow graph, spent time: 0.3648684024810791
Model saved in file: params/model.ckpt


  1%|          | 71/10000 [00:00<00:51, 193.27it/s]

...

  5%|▌         | 513/10000 [00:05<01:35, 98.96it/s]

497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515


  5%|▌         | 534/10000 [00:05<01:35, 98.70it/s]

516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534


535
536
537
538
539
540
541
542
Model saved in file: params/model.ckpt


In [32]:
class Test(object):
    def __getitem__(self, idx):
        return idx

In [33]:
test = Test()
x = test[::-1]

In [37]:
np.array(10)[x]

IndexError: too many indices for array

In [23]:
range(x.stop)

range(0, 10)

In [24]:
isinstance(x, slice)

True